<a href="https://colab.research.google.com/github/hholl19/Deep-Leaning-Class-Notes/blob/hholl19_added_things/Applied_Deep_Learning_Notebook_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import pandas as pd
import numpy as np
import os
import pathlib 
from pathlib import Path
import cv2
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dropout, Dense, Flatten, Conv2D, MaxPooling2D 
from keras.layers.normalization import BatchNormalization 
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

from numpy import expand_dims
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from matplotlib import pyplot

**Attempts to add dataset with Kaggle API and Google Colab. Main issue was unable to pull image files for model.**

In [ ]:
#!mkdir ~/.kaggle #create the .kaggle folder in your root directory
#!echo '{"username":"hholl19","key":"5209baad90fe207be2b471380de09e89"}' > ~/.kaggle/kaggle.json #write kaggle API credentials to kaggle.json
#!chmod 600 ~/.kaggle/kaggle.json  # set permissions
#!pip install kaggle 

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
#os.getcwd()

'/content'

In [ ]:
#pwd

'/content'

In [ ]:
#!kaggle datasets download -d {hackerearths-snakes-in-the-hood} -p /content [-w]
#!unzip /content/hackerearths-snakes-in-the-hood.zip -d review

404 - Not Found
Archive:  /content/hackerearths-snakes-in-the-hood.zip
replace review/dataset/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

**Wrote in Path directly to my Google Drive to Gain Access to full Dataset.**

In [2]:
file_path= '/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs'
folders = os.listdir(file_path)
print(folders)

['test.csv', 'train.csv', 'train', 'test']


In [3]:
df_train= pd.read_csv('/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train.csv')
df_train.head()

,image_id,breed
0,a8b3ad1dde,nerodia-erythrogaster
1,8b492b973d,pantherophis-vulpinus
2,929b99ea92,thamnophis-sirtalis
3,bbac7385e2,pantherophis-obsoletus
4,ef776b1488,agkistrodon-contortrix


The test set doesn't have any labels so we can't determine if our answers are correct or not. As a result, we will be splitting this dataset. 

In [4]:
df_train.shape

(5508, 2)

In [5]:
df_train.nunique()

image_id    5508
breed         35
dtype: int64

In [6]:
df_train['breed'].value_counts()

thamnophis-sirtalis            509
storeria-dekayi                412
pantherophis-obsoletus         366
crotalus-atrox                 354
pituophis-catenifer            298
nerodia-sipedon                259
agkistrodon-contortrix         206
lampropeltis-triangulum        197
diadophis-punctatus            184
pantherophis-alleghaniensis    172
nerodia-erythrogaster          171
lampropeltis-californiae       166
opheodrys-aestivus             152
agkistrodon-piscivorus         147
crotalus-horridus              142
coluber-constrictor            129
heterodon-platirhinos          118
thamnophis-proximus            112
nerodia-rhombifer              104
thamnophis-radix               100
pantherophis-guttatus          100
haldea-striatula                98
pantherophis-spiloides          92
rhinocheilus-lecontei           91
storeria-occipitomaculata       87
crotalus-viridis                83
masticophis-flagellum           83
thamnophis-marcianus            82
crotalus-ruber      

*Added in image file paths to pull images into CSV for easier processing.*

In [11]:
df_train["image_id"]="/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train/" + df_train["image_id"] + ".jpg"
df_train.head()

,image_id,breed
0,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,nerodia-erythrogaster
1,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,pantherophis-vulpinus
2,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,thamnophis-sirtalis
3,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,pantherophis-obsoletus
4,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,agkistrodon-contortrix


In [12]:
train_datagen = ImageDataGenerator(rescale=1/255,
        validation_split=0.2,
        data_format='channels_last',
        rotation_range=30,
        horizontal_flip=True,
        fill_mode='reflect') 
test_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [13]:
train_gen = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory="/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train/",
        x_col="image_id",
        y_col="breed",
        subset="training",
        batch_size=32,
        seed=123,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical", # this does the work of encoding
        target_size=(224,224)) # Change acc to model requirements

Found 4407 validated image filenames belonging to 35 classes.


In [14]:
test_gen = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory="/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train/",
        x_col="image_id",
        y_col="breed",
        subset="validation",
        batch_size=32,
        seed=123,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical", # this does the work of encoding
        target_size=(224,224))

Found 1101 validated image filenames belonging to 35 classes.


**Building out the Model->Used Alexnet structure for Model**


In [17]:
model = Sequential()

model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))

model.add(Dense(35, activation='softmax'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 10, 256)       1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 256)         5

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_gen, batch_size=64, epochs=100, verbose=1, shuffle=True)

Epoch 1/100
138/138 [==============================] - 294s 2s/step - loss: 5.8435 - accuracy: 0.0361
Epoch 2/100
138/138 [==============================] - 276s 2s/step - loss: 4.8850 - accuracy: 0.0502
Epoch 3/100
138/138 [==============================] - 273s 2s/step - loss: 4.7068 - accuracy: 0.0485
Epoch 4/100
138/138 [==============================] - 274s 2s/step - loss: 4.1808 - accuracy: 0.0613
Epoch 5/100
138/138 [==============================] - 272s 2s/step - loss: 4.0647 - accuracy: 0.0540
Epoch 6/100
138/138 [==============================] - 274s 2s/step - loss: 3.6661 - accuracy: 0.0720
Epoch 7/100
138/138 [==============================] - 273s 2s/step - loss: 3.7785 - accuracy: 0.0628
Epoch 8/100
138/138 [==============================] - 272s 2s/step - loss: 3.8939 - accuracy: 0.0656
Epoch 9/100
138/138 [==============================] - 269s 2s/step - loss: 3.6112 - accuracy: 0.0847
Epoch 10/100
138/138 [==============================] - 270s 2s/step - loss: 3.635

In [43]:
model.fit(train_datagen, batch_size=128, epochs=10, verbose=1, validation_data=(test_datagen))

Epoch 1/10
138/138 [==============================] - 297s 2s/step - loss: 3.8744 - accuracy: 0.0663 - val_loss: 3.9964 - val_accuracy: 0.0245
Epoch 2/10
138/138 [==============================] - 297s 2s/step - loss: 3.6872 - accuracy: 0.0733 - val_loss: 3.4584 - val_accuracy: 0.0781
Epoch 3/10
138/138 [==============================] - 296s 2s/step - loss: 3.6639 - accuracy: 0.0760 - val_loss: 6.5410 - val_accuracy: 0.0291
Epoch 4/10
138/138 [==============================] - 298s 2s/step - loss: 3.8606 - accuracy: 0.0676 - val_loss: 4.5831 - val_accuracy: 0.0936
Epoch 5/10
138/138 [==============================] - 296s 2s/step - loss: 3.7899 - accuracy: 0.0678 - val_loss: 6.2239 - val_accuracy: 0.0945
Epoch 6/10
138/138 [==============================] - 294s 2s/step - loss: 3.9329 - accuracy: 0.0581 - val_loss: 4.1075 - val_accuracy: 0.0272
Epoch 7/10
138/138 [==============================] - 295s 2s/step - loss: 3.6396 - accuracy: 0.0762 - val_loss: 3.5134 - val_accuracy: 0.0790

In [42]:
model.fit(train_datagen, steps_per_epoch=15, 
                    epochs=16, validation_data=test_datagen, 
                    validation_steps=15)

Epoch 1/16
15/15 [==============================] - 55s 4s/step - loss: 4.2716 - accuracy: 0.0604 - val_loss: 4.5001 - val_accuracy: 0.0200
Epoch 2/16
15/15 [==============================] - 90s 6s/step - loss: 4.3087 - accuracy: 0.0417 - val_loss: 5.6832 - val_accuracy: 0.0263
Epoch 3/16
15/15 [==============================] - 55s 4s/step - loss: 4.3561 - accuracy: 0.0437 - val_loss: 4.2368 - val_accuracy: 0.0300
Epoch 4/16
15/15 [==============================] - 54s 4s/step - loss: 4.4643 - accuracy: 0.0646 - val_loss: 4.4888 - val_accuracy: 0.0200
Epoch 5/16
15/15 [==============================] - 54s 4s/step - loss: 4.4845 - accuracy: 0.0417 - val_loss: 5.2646 - val_accuracy: 0.0336
Epoch 6/16
15/15 [==============================] - 54s 4s/step - loss: 4.6404 - accuracy: 0.0396 - val_loss: 4.5662 - val_accuracy: 0.0527
Epoch 7/16
15/15 [==============================] - 54s 4s/step - loss: 4.3778 - accuracy: 0.0667 - val_loss: 4.3404 - val_accuracy: 0.0536
Epoch 8/16
15/15 [==